In [31]:
import sys
import codecs
import numpy as np

### Challege 1
#### Convert Hex to Base64
Test string: 49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d

Ouput: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t

In [15]:
def hex_to_b64(bytes_Hex):
    hex_Decoded = codecs.decode(bytes_Hex, 'hex')
    b64_Encoded = codecs.encode(hex_Decoded, 'base64')
    return b64_Encoded

In [16]:
bstr_test = b'49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
print(hex_to_b64(bstr_test))

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'


### Challege 2
#### Fixed XOR
Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c
... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965
... should produce:

746865206b696420646f6e277420706c6179


In [17]:
def fixed_xor(byte_str1, byte_str2):
    hex_str1 = int(str(byte_str1.decode('utf-8')), base=16)
    hex_str2 = int(str(byte_str2.decode('utf-8')), base=16)
    val_XOR = hex(hex_str1 ^ hex_str2).encode()[2:]
    return val_XOR

In [18]:
byte_teststr1 = b'1c0111001f010100061a024b53535009181c'
byte_teststr2 = b'686974207468652062756c6c277320657965'
print(fixed_xor(byte_teststr1, byte_teststr2))

b'746865206b696420646f6e277420706c6179'


### Challege 3
#### Single-byte XOR Cypher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736
... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

In [21]:
def get_str_letter_freq(str_Test, dict_EnglishLetterFreq):
    """
    Input: string of letters, a dictionary containing the letters to be tested as keys
    Output: a dictionary of the frequency of letters in the string
    Notes: Called by EnglishScore
    """
    list_EnglishLetters = [i for i, k in dict_EnglishLetterFreq.items()]
    dict_StringLetterCount = dict.fromkeys(list_EnglishLetters , 0)
    str_Test = str_Test.lower()
    val_NotLetterPenalty = 0
    for c in str_Test:
        if c in list_EnglishLetters:
            dict_StringLetterCount[c] += 1
        elif c == ' ':
            val_NotLetterPenalty += 0 #no penalty for spaces
        else:
            val_NotLetterPenalty += 0.1 #adjustable penalty for non-English letter, non-space characters
    dict_StringLetterFreq = {k: v / len(str_Test) for k, v in dict_StringLetterCount.items()}
    return dict_StringLetterFreq, val_NotLetterPenalty

def test_str_for_engish(str_Test):
    dict_EnglishLetterFreq = {'a': 0.0817,'b': 0.0149,'c': 0.0278,'d': 0.0425,'e': 0.127,'f': 0.0223,'g': 0.0202,
                              'h': 0.0609,'i': 0.0697,'j': 0.0015,'k': 0.0077,'l': 0.0403,'m': 0.0241,'n': 0.0675,
                              'o': 0.0751,'p': 0.0193,'q': 0.001,'r': 0.0599,'s': 0.0633,'t': 0.0906,'u': 0.0276,
                              'v': 0.0098,'w': 0.0236,'x': 0.0015, 'y': 0.0197,'z': 0.0007}
    dict_TestLetterFreq, val_NotLetterPenalty = get_str_letter_freq(str_Test, dict_EnglishLetterFreq)
    list_EnglishScore = [(a - b)**2 for (a,b) in zip(dict_TestLetterFreq.values(), dict_EnglishLetterFreq.values())]
    total_EnglishScore = sum(list_EnglishScore) + val_NotLetterPenalty
    return total_EnglishScore

def score_english_strings(list_TestStrings):
    dict_StringScores = dict.fromkeys(list_TestStrings, 0)
    for s in list_TestStrings:
        dict_StringScores[s] = test_str_for_engish(s)
    string_MinScore = min(dict_StringScores, key=lambda key: dict_StringScores[key])
    return string_MinScore, dict_StringScores[string_MinScore], dict_StringScores

In [58]:
def single_byte_xor(str_Encoded, byte_Char):
    list_Decoded = []
    for c in str_Encoded:
        val_XOR = c ^ byte_Char
        list_Decoded.append(chr(val_XOR))
    str_Decoded = ''.join(list_Decoded)
    return str_Decoded


def decode_single_byte_xor(str_HexEncode):
    list_DecodedStrings = []
    str_HexDecode = codecs.decode(str_HexEncode, 'hex')
    for i in range(0, 256):
        bytes_XORd = single_byte_xor(str_HexDecode, i)
        if len(bytes_XORd) > 0:
            list_DecodedStrings.append(bytes_XORd)
    return list_DecodedStrings

In [59]:
str_SingleByteEncode = b'1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
list_DecodedStrings = decode_single_byte_xor(str_SingleByteEncode)
str_BestEngScore, val_BestEngScore, dict_AllScores = score_english_strings(list_DecodedStrings)
print(str_BestEngScore)

Cooking MC's like a pound of bacon


In [60]:
#list_DecodedStrings

### Challenge 4
#### Detect single-character XOR
One of the 60-character strings in this file has been encrypted by single-character XOR.

In [61]:
f = open('./ChallengeSet1_4.txt', 'rb')
list_EncodedStrings = [l.strip() for l in f]

In [62]:
list_AllDecodedStrings = []
for s in list_EncodedStrings:
    
    list_DecodedElement = DecodeSingleByteXOR(s)
    
    if len(list_DecodedElement) > 0:
        str_BestEngScore, val_BestEngScore, dict_AllScores = ScoreStringList(list_DecodedElement)
        list_AllDecodedStrings.extend(list_DecodedElement)

In [63]:
str_BestEngScore, val_BestEngScore, dict_AllScores = ScoreStringList(list_AllDecodedStrings)
print(str_BestEngScore)

Now that the party is jumping



In [13]:
#[(k, dict_AllScores[k]) for k in sorted(dict_AllScores, key = dict_AllScores.get)]

### Challenge 5
#### Implement repeating-key XOR
Here is the opening stanza of an important work of the English language:

>Burning 'em, if you ain't quick and nimble  
I go crazy when I hear a cymbal  

Encrypt it, under the key "ICE", using repeating-key XOR.  

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:

>0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272  
a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f

Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.

In [131]:
def encode_repeating_key_XOR(str_Encode, str_Key):
    list_XORdChars = []
    val_KeyLength = len(str_Key)
    for i in range(0, len(str_Encode)):
        val_XOR = str_Key[i % val_KeyLength] ^  str_Encode[i]
        list_XORdChars.append(hex(val_XOR)[2:].zfill(2))
    return ''.join(list_XORdChars)

In [133]:
str_Encode =  b"Burning 'em, if you ain't quick and nimble I go crazy when I hear a cymbal"
str_Key = b'ICE'

print(encode_repeating_key_XOR(str_Encode, str_Key))

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20690a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


### Challenge 6
#### Break repeating-key XOR

There's a file here. It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
>this is a test

and
>wokka wokka!!!

is 37.  

Make sure your code agrees before you proceed.  

For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.  

The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.  

Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.  

Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.  

Solve each block as if it was single-character XOR. You already have code to do this.  

For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

In [103]:
def str_to_bits(s):
    bits_str = [bin(ord(c))[2:].zfill(8).encode() for c in s]
    return bits_str

def hamming_distance(str1, str2):
    assert len(str1) == len(str2), 'String lengths are not equal'
    list_bits1 = str_to_bits(str1)
    list_bits2 = str_to_bits(str2)
    total_diffbits = 0
    for (bits1, bits2) in zip(list_bits1, list_bits2):
        i = 0
        n = 0
        for c in bits1:
            if c!= bits2[i]:
                n += 1
            i += 1
        total_diffbits += n
    return total_diffbits


In [105]:
str1 = 'this is a test'
str2  = 'wokka wokka!!!'
hamming_distance(str1, str2)

37

In [134]:
f = open('./ChallengeSet1_6.txt', 'rb')
list_EncodedStrings = [l.strip() for l in f]

In [135]:
list_EncodedStrings

[b'HUIfTQsPAh9PE048GmllH0kcDk4TAQsHThsBFkU2AB4BSWQgVB0dQzNTTmVS',
 b'BgBHVBwNRU0HBAxTEjwMHghJGgkRTxRMIRpHKwAFHUdZEQQJAGQmB1MANxYG',
 b'DBoXQR0BUlQwXwAgEwoFR08SSAhFTmU+Fgk4RQYFCBpGB08fWXh+amI2DB0P',
 b'QQ1IBlUaGwAdQnQEHgFJGgkRAlJ6f0kASDoAGhNJGk9FSA8dDVMEOgFSGQEL',
 b'QRMGAEwxX1NiFQYHCQdUCxdBFBZJeTM1CxsBBQ9GB08dTnhOSCdSBAcMRVhI',
 b'CEEATyBUCHQLHRlJAgAOFlwAUjBpZR9JAgJUAAELB04CEFMBJhAVTQIHAh9P',
 b'G054MGk2UgoBCVQGBwlTTgIQUwg7EAYFSQ8PEE87ADpfRyscSWQzT1QCEFMa',
 b'TwUWEXQMBk0PAg4DQ1JMPU4ALwtJDQhOFw0VVB1PDhxFXigLTRkBEgcKVVN4',
 b'Tk9iBgELR1MdDAAAFwoFHww6Ql5NLgFBIg4cSTRWQWI1Bk9HKn47CE8BGwFT',
 b'QjcEBx4MThUcDgYHKxpUKhdJGQZZVCFFVwcDBVMHMUV4LAcKQR0JUlk3TwAm',
 b'HQdJEwATARNFTg5JFwQ5C15NHQYEGk94dzBDADsdHE4UVBUaDE5JTwgHRTkA',
 b'Umc6AUETCgYAN1xGYlUKDxJTEUgsAA0ABwcXOwlSGQELQQcbE0c9GioWGgwc',
 b'AgcHSAtPTgsAABY9C1VNCAINGxgXRHgwaWUfSQcJABkRRU8ZAUkDDTUWF01j',
 b'OgkRTxVJKlZJJwFJHQYADUgRSAsWSR8KIgBSAAxOABoLUlQwW1RiGxpOCEtU',
 b'YiROCk8gUwY1C1IJCAACEU8QRSxORTBSHQYGTlQJC1lOBAAXRTpCUh0FDxhU',
 b'ZXhzLFt